In [2]:
import pandas as pd

In [3]:
output_steamspy = pd.read_csv('output_steamspy.csv')
steamstore = pd.read_csv('steam_store_data_2024.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [4]:
train

,id,app_id,content,author_id,is_positive
0,1,340,GG,76561198152563064,1
1,2,630,its fun,76561198006953365,1
2,3,730,POKAZUJE ZE MAM PIERDOLONE 15 PINGU A CALY CZA...,76561199082720424,0
3,4,730,this game suck its not the game its self its t...,76561198305268228,0
4,5,30,Kind of difficult to play without the source u...,76561198207922887,0
...,...,...,...,...,...
140800,140801,10,я стрелял меня насиловали,76561198412836420,1
140801,140802,240,This game is gold\n,76561199335981839,1
140802,140803,550,"The best zombie horde shooter to this day, I w...",76561199030428167,1
140803,140804,570,vvvvvv,76561199229539481,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140805 entries, 0 to 140804
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           140805 non-null  int64 
 1   app_id       140805 non-null  int64 
 2   content      140495 non-null  object
 3   author_id    140805 non-null  int64 
 4   is_positive  140805 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 5.4+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60346 entries, 0 to 60345
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         60346 non-null  int64 
 1   app_id     60346 non-null  int64 
 2   content    60228 non-null  object
 3   author_id  60346 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ MB


In [7]:
#replace Nan in test.content with empty string
test['content'] = test['content'].fillna('')

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60346 entries, 0 to 60345
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         60346 non-null  int64 
 1   app_id     60346 non-null  int64 
 2   content    60346 non-null  object
 3   author_id  60346 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ MB


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

# Загрузка данных
df = train.copy()
df = df.dropna(subset=['content'])    # Удалим строки с пустым контентом

# Выбор признаков и метки
X = df['content']
y = df['is_positive']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование текста в TF-IDF векторы
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), decode_error="strict")  # можно поиграть с параметрами
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [8]:
# Обучение модели логистической регрессии
#model = LogisticRegression(max_iter=1000)
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train_tfidf, y_train)

# Предсказание и оценка
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7616997046158226
              precision    recall  f1-score   support

           0       0.86      0.61      0.71     13686
           1       0.71      0.90      0.80     14413

    accuracy                           0.76     28099
   macro avg       0.78      0.76      0.75     28099
weighted avg       0.78      0.76      0.76     28099



In [54]:
model1 = LogisticRegression(max_iter=1000)
model1.fit(X_train_tfidf, y_train)

y_pred1 = model1.predict(X_test_tfidf)
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

Accuracy (Logistic Regression): 0.851916438307413
              precision    recall  f1-score   support

           0       0.88      0.81      0.84     13686
           1       0.83      0.89      0.86     14413

    accuracy                           0.85     28099
   macro avg       0.85      0.85      0.85     28099
weighted avg       0.85      0.85      0.85     28099



In [57]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model2.fit(X_train_tfidf, y_train)

y_pred2 = model2.predict(X_test_tfidf)
print("Accuracy (Random Forest):", accuracy_score(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

Accuracy (Random Forest): 0.8298160076871063
              precision    recall  f1-score   support

           0       0.85      0.79      0.82     13686
           1       0.82      0.86      0.84     14413

    accuracy                           0.83     28099
   macro avg       0.83      0.83      0.83     28099
weighted avg       0.83      0.83      0.83     28099



In [55]:
# используем модель для предсказания на тестовых данных
X_test_final = test['content']
X_test_final_tfidf = vectorizer.transform(X_test_final)

# Предсказание на тестовых данных
y_test_final_pred = model.predict(X_test_final_tfidf)
# Сохранение предсказаний в файл

submission = pd.DataFrame({
    'id': test['id'],
    'sentiment': y_test_final_pred
})

In [56]:
submission.to_csv('submission.csv', index=False)

___

In [9]:
import pandas as pd
from gensim.models import Word2Vec, FastText
import re

# Очистка и токенизация
def preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text.split()

df_ft = train.copy()

# Выбор признаков и метки
X = df['content'] 
y = df['is_positive']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Токенизация
X_train_tokens = X_train.apply(preprocess)
X_test_tokens = X_test.apply(preprocess)


In [10]:
ft_model = FastText(sentences=X_train_tokens, vector_size=200, window=5, min_count=1, sg=1, epochs=10)

# Преобразование текстов в векторы



In [11]:
import numpy as np

def vectorize_text(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Преобразование текста в векторы
X_train_vectors = np.array([vectorize_text(tokens, ft_model) for tokens in X_train_tokens])
X_test_vectors = np.array([vectorize_text(tokens, ft_model) for tokens in X_test_tokens])



In [12]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_vectors, y_train)
y_pred_lr = model_lr.predict(X_test_vectors)
print("Accuracy (Logistic Regression with FastText):", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Accuracy (Logistic Regression with FastText): 0.819993594078081
              precision    recall  f1-score   support

           0       0.81      0.82      0.82     13686
           1       0.83      0.82      0.82     14413

    accuracy                           0.82     28099
   macro avg       0.82      0.82      0.82     28099
weighted avg       0.82      0.82      0.82     28099



___

In [13]:
from sklearn.pipeline import Pipeline

In [14]:
train = pd.read_csv('train.csv')
train['app_id'] = train['app_id'].astype(str)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140805 entries, 0 to 140804
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           140805 non-null  int64 
 1   app_id       140805 non-null  object
 2   content      140495 non-null  object
 3   author_id    140805 non-null  int64 
 4   is_positive  140805 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.4+ MB


In [15]:
train['content'] = train['content'].fillna('')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140805 entries, 0 to 140804
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           140805 non-null  int64 
 1   app_id       140805 non-null  object
 2   content      140805 non-null  object
 3   author_id    140805 non-null  int64 
 4   is_positive  140805 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.4+ MB


In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize


def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

def get_word_count(x):
    return np.array([len(word_tokenize(t)) for t in x]).reshape(-1, 1)

def get_unique_word_count(x):
    return np.array([len(set(word_tokenize(t))) for t in x]).reshape(-1, 1)

def mean_word_length(x):
    return np.array([np.mean([len(w) for w in word_tokenize(t)]) for t in x]).reshape(-1, 1)

def mean_syllable_count(x):
    return np.array([np.mean([len(list(filter(lambda x: x.lower() in 'aeiouy', w))) for w in word_tokenize(t)]) for t in x]).reshape(-1, 1)

def get_sentences_count(x):
    return np.array([len(x.split('.')) for x in x]).reshape(-1, 1)

def vocabulary_size(x):
    return np.array([len(set(word_tokenize(x))) for x in x]).reshape(-1, 1)

train['text_length'] = get_text_length(train['content'])
train['word_count'] = get_word_count(train['content'])
train['unique_word_count'] = get_unique_word_count(train['content'])
train['mean_word_length'] = mean_word_length(train['content'])
train['mean_syllable_count'] = mean_syllable_count(train['content'])
train['sentences_count'] = get_sentences_count(train['content'])
train['vocabulary_size'] = vocabulary_size(train['content'])

c:\GitHub\spbu-ml\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\GitHub\spbu-ml\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\GitHub\spbu-ml\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\GitHub\spbu-ml\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [17]:
train

,id,app_id,content,author_id,is_positive,text_length,word_count,unique_word_count,mean_word_length,mean_syllable_count,sentences_count,vocabulary_size
0,1,340,GG,76561198152563064,1,2,1,1,2.000000,0.000000,1,1
1,2,630,its fun,76561198006953365,1,7,2,2,3.000000,1.000000,1,2
2,3,730,POKAZUJE ZE MAM PIERDOLONE 15 PINGU A CALY CZA...,76561199082720424,0,94,17,16,4.588235,2.058824,1,16
3,4,730,this game suck its not the game its self its t...,76561198305268228,0,224,44,33,4.113636,1.613636,1,33
4,5,30,Kind of difficult to play without the source u...,76561198207922887,0,86,19,17,3.789474,1.421053,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...
140800,140801,10,я стрелял меня насиловали,76561198412836420,1,25,4,4,5.500000,0.000000,1,4
140801,140802,240,This game is gold\n,76561199335981839,1,18,4,4,3.500000,1.250000,1,4
140802,140803,550,"The best zombie horde shooter to this day, I w...",76561199030428167,1,80,16,15,4.125000,1.687500,1,15
140803,140804,570,vvvvvv,76561199229539481,0,6,1,1,6.000000,0.000000,1,1


In [18]:
#replace NaN values with 0
train = train.fillna(0)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140805 entries, 0 to 140804
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   140805 non-null  int64  
 1   app_id               140805 non-null  object 
 2   content              140805 non-null  object 
 3   author_id            140805 non-null  int64  
 4   is_positive          140805 non-null  int64  
 5   text_length          140805 non-null  int32  
 6   word_count           140805 non-null  int32  
 7   unique_word_count    140805 non-null  int32  
 8   mean_word_length     140805 non-null  float64
 9   mean_syllable_count  140805 non-null  float64
 10  sentences_count      140805 non-null  int32  
 11  vocabulary_size      140805 non-null  int32  
dtypes: float64(2), int32(5), int64(3), object(2)
memory usage: 10.2+ MB


In [19]:
X = train.drop(columns=['is_positive', 'id', 'author_id'])
y = train['is_positive']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train

,app_id,content,text_length,word_count,unique_word_count,mean_word_length,mean_syllable_count,sentences_count,vocabulary_size
66140,240,bought this game ages ago for gmod textures,43,8,8,4.500000,1.750000,1,8
75301,240,yes,3,1,1,3.000000,2.000000,1,1
115162,620,please gieve refund frined gave game and i wan...,109,20,17,4.500000,2.000000,1,17
22128,280,---{Graphics}---\n☐ You forget what reality is...,1341,341,165,3.152493,1.126100,2,165
55317,730,děkuji dotal jsem penalizaci za dřívější leavn...,90,13,13,6.000000,1.923077,1,13
...,...,...,...,...,...,...,...,...,...
110268,440,Yerp.,5,2,2,2.500000,1.000000,2,2
119879,30,"Has an older community in it, not much young b...",286,64,48,3.640625,1.312500,4,48
103694,300,eh.,3,2,2,1.500000,0.500000,2,2
131932,70,ew (i have a headache),22,7,7,2.571429,1.285714,1,7


In [116]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

# Create a preprocessing pipeline that will handle both text and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=10000, ngram_range=(1, 2)), 'content'),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['app_id']),  # Use a list for column names
        ('num', StandardScaler(), ['text_length', 'word_count', 'unique_word_count', 
                                    'mean_word_length', 'mean_syllable_count', 
                                    'sentences_count', 'vocabulary_size'])
    ],
    remainder='drop'
)

# Create the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_features=10000,
                                                                  ngram_range=(1,
                                                                               2)),
                                                  'content'),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['app_id']),
                                                 ('num', StandardScaler(),
                                                  ['text_length', 'word_count',
                                                   'unique_word_count',
                                                   'mean_word_length',
                                                   'mean_syllable_count',
                                                   'sentences_count',
                                                   'vocabulary_size'])])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

In [118]:
y_val_pred = pipeline.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
y_val_report = classification_report(y_val, y_val_pred)
print("Validation Classification Report:\n", y_val_report)

Validation Accuracy: 0.8707787365505486
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.79      0.86     13648
           1       0.83      0.95      0.88     14513

    accuracy                           0.87     28161
   macro avg       0.88      0.87      0.87     28161
weighted avg       0.88      0.87      0.87     28161



___

In [ ]:
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer

train = pd.read_csv('train.csv')
train['app_id'] = train['app_id'].astype(str)
train['content'] = train['content'].fillna('')

# Очистка текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

train['content'] = train['content'].astype(str).apply(clean_text)



def get_text_length(x): return np.array([len(t) for t in x]).reshape(-1, 1)
def get_word_count(x): return np.array([len(word_tokenize(t)) for t in x]).reshape(-1, 1)
def get_unique_word_count(x): return np.array([len(set(word_tokenize(t))) for t in x]).reshape(-1, 1)
def mean_word_length(x): return np.array([np.mean([len(w) for w in word_tokenize(t)]) for t in x]).reshape(-1, 1)
def mean_syllable_count(x): return np.array([
    np.mean([sum(1 for c in w if c.lower() in 'aeiouyаеёиоуыэюя') for w in word_tokenize(t)]) for t in x
]).reshape(-1, 1)
def get_sentences_count(x): return np.array([len(re.split(r'[.!?]', t)) for t in x]).reshape(-1, 1)
def vocabulary_size(x): return np.array([len(set(word_tokenize(t))) for t in x]).reshape(-1, 1)

train['text_length'] = get_text_length(train['content'])
train['word_count'] = get_word_count(train['content'])
train['unique_word_count'] = get_unique_word_count(train['content'])
train['mean_word_length'] = mean_word_length(train['content'])
train['mean_syllable_count'] = mean_syllable_count(train['content'])
train['sentences_count'] = get_sentences_count(train['content'])
train['vocabulary_size'] = vocabulary_size(train['content'])

train = train.fillna(0)



X = train.drop(columns=['is_positive', 'id', 'author_id'])
y = train['is_positive']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=15000, ngram_range=(1, 2), stop_words='english'), 'content'),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['app_id']),
        ('num', StandardScaler(), ['text_length', 'word_count', 'unique_word_count',
                                   'mean_word_length', 'mean_syllable_count',
                                   'sentences_count', 'vocabulary_size'])
    ],
    remainder='drop'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=4,
        random_state=42
    ))
])



pipeline.fit(X_train, y_train)

y_val_pred = pipeline.predict(X_val)
acc = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification report:")
print(classification_report(y_val, y_val_pred))


___

In [31]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Загрузка данных
train = pd.read_csv('train.csv')
train['app_id'] = train['app_id'].astype(str)
train['content'] = train['content'].fillna('')

# Очистка текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

train['content'] = train['content'].astype(str).apply(clean_text)

# Дополнительные фичи
def get_text_length(x): return np.array([len(t) for t in x]).reshape(-1, 1)
def get_word_count(x): return np.array([len(word_tokenize(t)) for t in x]).reshape(-1, 1)
def get_unique_word_count(x): return np.array([len(set(word_tokenize(t))) for t in x]).reshape(-1, 1)
def mean_word_length(x): return np.array([np.mean([len(w) for w in word_tokenize(t)]) if len(word_tokenize(t)) > 0 else 0 for t in x]).reshape(-1, 1)
def mean_syllable_count(x): return np.array([np.mean([len(list(filter(lambda x: x.lower() in 'aeiouy', w))) for w in word_tokenize(t)]) if len(word_tokenize(t)) > 0 else 0 for t in x]).reshape(-1, 1)
def get_sentences_count(x): return np.array([len(x.split('.')) for x in x]).reshape(-1, 1)
def vocabulary_size(x): return np.array([len(set(word_tokenize(x))) for x in x]).reshape(-1, 1)
def count_exclamations(x): return np.array([t.count('!') for t in x]).reshape(-1, 1)
def count_questions(x): return np.array([t.count('?') for t in x]).reshape(-1, 1)
def count_not(x): return np.array([t.lower().count('not') for t in x]).reshape(-1, 1)
def count_but(x): return np.array([t.lower().count('but') for t in x]).reshape(-1, 1)

# Добавление фичей
train['text_length'] = get_text_length(train['content'])
train['word_count'] = get_word_count(train['content'])
train['unique_word_count'] = get_unique_word_count(train['content'])
train['mean_word_length'] = mean_word_length(train['content'])
train['mean_syllable_count'] = mean_syllable_count(train['content'])
train['sentences_count'] = get_sentences_count(train['content'])
train['vocabulary_size'] = vocabulary_size(train['content'])
train['count_exclamations'] = count_exclamations(train['content'])
train['count_questions'] = count_questions(train['content'])
train['count_not'] = count_not(train['content'])
train['count_but'] = count_but(train['content'])

# Целевая переменная и разбиение
X = train.drop(columns=['is_positive', 'id', 'author_id'])
y = train['is_positive']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Подготовка TF-IDF векторайзеров
word_vectorizer = ('word_tfidf', TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    stop_words='english',
    analyzer='word'
), 'content')

char_vectorizer = ('char_tfidf', TfidfVectorizer(
    max_features=5000,
    ngram_range=(3, 5),
    analyzer='char_wb'
), 'content')

# Числовые признаки
numeric_features = [
    'text_length', 'word_count', 'unique_word_count',
    'mean_word_length', 'mean_syllable_count',
    'sentences_count', 'vocabulary_size',
    'count_exclamations', 'count_questions',
    'count_not', 'count_but'
]


In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB


# Предобработка
preprocessor = ColumnTransformer(
    transformers=[
        word_vectorizer,
        char_vectorizer,
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['app_id']),
        ('num', MinMaxScaler(), numeric_features)
    ],
    remainder='drop'
)

# Базовые классификаторы для ансамбля
estimators = [
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('nb', MultinomialNB())  # NB работает только на неотрицательных данных — TF-IDF
]

# Ансамбль
stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    passthrough=False,
    n_jobs=-1
)

# Полный пайплайн
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', stacking)
])

# Обучение
#pipeline.fit(X_train, y_train)
#pipeline.fit(X, y)

from joblib import load
pipeline = load('C:\GitHub\spbu-ml\spbu_ml_2025\homeworks\games_challenge\data/text_classification_pipeline.pkl')

# Оценка
y_pred = pipeline.predict(X)
print("Validation Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:\n", classification_report(y, y_pred))


Validation Accuracy: 0.9446965661730763
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94     68938
           1       0.94      0.96      0.95     71867

    accuracy                           0.94    140805
   macro avg       0.95      0.94      0.94    140805
weighted avg       0.94      0.94      0.94    140805



In [ ]:
test = pd.read_csv('test.csv')
test['app_id'] = test['app_id'].astype(str)
test['content'] = test['content'].fillna('')

# Очистка текста
test['content'] = test['content'].astype(str).apply(clean_text)

# Добавление тех же фичей, что и в тренировочном наборе
test['text_length'] = get_text_length(test['content'])
test['word_count'] = get_word_count(test['content'])
test['unique_word_count'] = get_unique_word_count(test['content'])
test['mean_word_length'] = mean_word_length(test['content'])
test['mean_syllable_count'] = mean_syllable_count(test['content'])
test['sentences_count'] = get_sentences_count(test['content'])
test['vocabulary_size'] = vocabulary_size(test['content'])
test['count_exclamations'] = count_exclamations(test['content'])
test['count_questions'] = count_questions(test['content'])
test['count_not'] = count_not(test['content'])
test['count_but'] = count_but(test['content'])

# Преобразование тестовых данных
X_test_final = test.drop(columns=['id', 'author_id'])

# Применение пайплайна к тестовым данным
y_test_final_pred = pipeline.predict(X_test_final)

# Сохранение предсказаний в файл
submission = pd.DataFrame({
    'id': test['id'],
    'sentiment': y_test_final_pred
})

submission.to_csv('submission.csv', index=False)
# Сохранение модели
import joblib
joblib.dump(pipeline, 'text_classification_pipeline.pkl')


['text_classification_pipeline.pkl']